In [14]:
import sys
print(sys.executable)
print(sys.version)


d:\akriti_project\anaconda_jupyternotebook\envs\rec_env\python.exe
3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]


In [15]:
!pip install pandas numpy seaborn matplotlib scipy scikit-surprise


In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


DATA_PATH = r"D:\Recommendation_system\online_course_recommendation_v2.csv"  # <-- change if needed
df = pd.read_csv(DATA_PATH)

df.shape
df.columns
df.head()
df.info()
df.isnull().sum()
df.describe()
df.duplicated().sum()
df.drop_duplicates()
df.nunique()
df['course_name'].unique()
df.dropna()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   user_id                   100000 non-null  int64  
 1   course_id                 100000 non-null  int64  
 2   course_name               100000 non-null  object 
 3   instructor                100000 non-null  object 
 4   course_duration_hours     100000 non-null  float64
 5   certification_offered     100000 non-null  object 
 6   difficulty_level          100000 non-null  object 
 7   rating                    100000 non-null  float64
 8   enrollment_numbers        100000 non-null  int64  
 9   course_price              100000 non-null  float64
 10  feedback_score            100000 non-null  float64
 11  study_material_available  100000 non-null  object 
 12  time_spent_hours          100000 non-null  float64
 13  previous_courses_taken    100000 non-null  in

,user_id,course_id,course_name,instructor,course_duration_hours,certification_offered,difficulty_level,rating,enrollment_numbers,course_price,feedback_score,study_material_available,time_spent_hours,previous_courses_taken
0,15796,9366,Python for Beginners,Emma Harris,39.1,Yes,Beginner,5.0,21600,317.50,0.797,Yes,17.60,4
1,861,1928,Cybersecurity for Professionals,Alexander Young,36.3,Yes,Beginner,4.3,15379,40.99,0.770,Yes,28.97,9
2,38159,9541,DevOps and Continuous Deployment,Dr. Mia Walker,13.4,Yes,Beginner,3.9,6431,380.81,0.772,Yes,52.44,4
3,44733,3708,Project Management Fundamentals,Benjamin Lewis,58.3,Yes,Beginner,3.1,48245,342.80,0.969,No,22.29,6
4,11285,3361,Ethical Hacking Masterclass,Daniel White,30.8,Yes,Beginner,2.8,34556,381.01,0.555,Yes,22.01,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,10647,5466,Graphic Design with Canva,Emma Harris,17.3,Yes,Beginner,3.9,49101,485.34,0.838,Yes,16.35,3
99996,13800,2623,Stock Market and Trading Strategies,Dr. John Smith,68.4,Yes,Beginner,3.5,35107,162.80,0.922,Yes,41.94,7
99997,47131,1556,Networking and System Administration,Dr. John Smith,73.8,Yes,Beginner,4.3,12146,24.02,0.990,Yes,15.87,5
99998,49654,6001,Graphic Design with Canva,Daniel White,30.3,Yes,Beginner,3.5,9933,402.24,0.630,Yes,21.05,4


Memory Optimization

In [17]:
mem_before = df.memory_usage(deep=True).sum() / 1024**2
print("Memory Before:", mem_before)

df_optimized = df.copy()

for col in df_optimized.select_dtypes(include=['int64']).columns:
    df_optimized[col] = pd.to_numeric(df_optimized[col], downcast='integer')

for col in df_optimized.select_dtypes(include=['float64']).columns:
    df_optimized[col] = pd.to_numeric(df_optimized[col], downcast='float')

for col in df_optimized.select_dtypes(include=['object']).columns:
    if df_optimized[col].nunique() / len(df_optimized[col]) < 0.5:
        df_optimized[col] = df_optimized[col].astype('category')

mem_after = df_optimized.memory_usage(deep=True).sum() / 1024**2
print("Memory After:", mem_after)

df = df_optimized


Memory Before: 39.55427932739258
Memory After: 3.4381303787231445


7️⃣ Exploratory Data Analysis (EDA)

In [18]:
numerical_cols = df.select_dtypes(include=['int64','float64']).columns

for col in numerical_cols:
    plt.figure(figsize=(8,4))
    sns.histplot(df[col], kde=True)
    plt.title(f"Distribution of {col}")
    plt.show()



8️⃣ Correlation Analysis

In [19]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

if len(numerical_cols) == 0:
    print("No numerical columns found.")
else:
    corr_matrix = df[numerical_cols].corr()

    if corr_matrix.empty:
        print("Correlation matrix is empty.")
    else:
        plt.figure(figsize=(10,8))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
        plt.show()



No numerical columns found.


Outlier Removal (Z-Score)

In [20]:
from scipy.stats import zscore
import numpy as np

numerical_cols = df.select_dtypes(include=['int64','float64']).columns

z_scores = np.abs(zscore(df[numerical_cols]))

outliers = (z_scores > 3).any(axis=1)

print("Outliers detected:", outliers.sum())

df_clean = df[~outliers]



Outliers detected: 0


Re-ranking

In [ ]:
# -------------------------------
# Normalize engagement features
# -------------------------------
df_clean['normalized_enrollment'] = (
    df_clean['enrollment_numbers'] - df_clean['enrollment_numbers'].min()
) / (df_clean['enrollment_numbers'].max() - df_clean['enrollment_numbers'].min())

df_clean['normalized_feedback'] = (
    df_clean['feedback_score'] - df_clean['feedback_score'].min()
) / (df_clean['feedback_score'].max() - df_clean['feedback_score'].min())


Prepare Data for Collaborative Filtering (CORE STEP)

In [21]:
ratings_df = df_clean[['user_id', 'course_id', 'rating']]
ratings_df.head()


,user_id,course_id,rating
0,15796,9366,5.0
1,861,1928,4.3
2,38159,9541,3.9
3,44733,3708,3.1
4,11285,3361,2.8


1️⃣1️⃣ Build Collaborative Filtering Model (SVD)

In [22]:
from surprise import Dataset, Reader, SVD
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df, reader)

trainset = data.build_full_trainset()



Train SVD Model (RAM-SAFE SETTINGS)

In [23]:
svd_model = SVD(
    n_factors=20,     # LOW RAM
    n_epochs=20,
    random_state=42
)

svd_model.fit(trainset)


Recommendation Function (USER-BASED)

In [ ]:
def recommend_courses(user_id, n_recommendations=5):
    all_courses = df_clean['course_id'].unique()
    taken_courses = df_clean[df_clean['user_id'] == user_id]['course_id'].unique()

    unseen_courses = [c for c in all_courses if c not in taken_courses]

    predictions = []

    for course_id in unseen_courses:
        pred = svd_model.predict(user_id, course_id)

        course_row = df_clean[df_clean['course_id'] == course_id].iloc[0]

        final_score = (
            0.7 * pred.est +
            0.2 * course_row['normalized_feedback'] +
            0.1 * course_row['normalized_enrollment']
        )

        predictions.append((course_id, final_score))

    # Sort by FINAL SCORE (re-ranked)
    predictions.sort(key=lambda x: x[1], reverse=True)

    top_course_ids = [c[0] for c in predictions[:n_recommendations]]

    result = (
        df_clean[df_clean['course_id'].isin(top_course_ids)]
        .drop_duplicates(subset='course_id')
        .head(n_recommendations)
        [['course_id', 'course_name', 'instructor',
          'difficulty_level', 'course_price']]
        .reset_index(drop=True)
    )

    return result


Test the Recommendation System

In [ ]:
# recommend_courses(user_id=1023, n_recommendations=5)


,course_id,course_name,instructor,difficulty_level,course_price
0,7040,Graphic Design with Canva,Alexander Young,Beginner,258.299988
1,1770,Fitness and Nutrition Coaching,Dr. Robert Davis,Beginner,230.119995
2,9376,Public Speaking Mastery,Dr. John Smith,Beginner,267.750000
3,7490,DevOps and Continuous Deployment,Dr. Mia Walker,Beginner,415.380005
4,1770,Fundamentals of Digital Marketing,Alexander Young,Intermediate,435.579987
5,8646,Cloud Computing Essentials,Prof. Emily Johnson,Advanced,60.619999
6,8646,Ethical Hacking Masterclass,Dr. Robert Davis,Advanced,20.040001
7,9376,Networking and System Administration,Olivia Taylor,Intermediate,257.399994
8,1770,AI for Business Leaders,James Clark,Beginner,144.259995
9,7040,Stock Market and Trading Strategies,Liam Adams,Intermediate,179.889999


Evaluation Metrics (Precision@K, Recall@K)

In [26]:
def precision_at_k(pred, actual, k=5):
    return len(set(pred[:k]) & set(actual)) / k

def recall_at_k(pred, actual, k=5):
    return len(set(pred[:k]) & set(actual)) / len(actual)

pred = [101,102,103,104,105]
actual = {101,110,103}

print("Precision@5:", precision_at_k(pred, actual))
print("Recall@5:", recall_at_k(pred, actual))


Precision@5: 0.4
Recall@5: 0.6666666666666666


(Optional) Launch Streamlit App

In [32]:
import subprocess

app_path = r"D:\Recommendation_system\app_svd.py"
subprocess.Popen(["streamlit", "run", app_path])


<Popen: returncode: None args: ['streamlit', 'run', 'D:\\Recommendation_syst...>

In [ ]:
'''Conclusion and Recommendations
Overall, our analysis has given us a clear and solid foundation for building a strong recommendation system. We started by cleaning and transforming 
the dataset, converting all categorical values into meaningful numbers and applying scaling so that every feature is treated fairly. Through our 
exploratory analysis, we gained a deeper understanding of what truly influences user choices—such as course ratings, difficulty levels, and learning
format—helping us see the patterns that matter. With this groundwork in place, we now have a well-prepared dataset ready for machine learning models.
Moving forward, we recommend enriching the system with real user behavior signals like clicks, time spent, and course completion, as these reflect 
genuine user interest. It will be valuable to experiment with multiple modeling approaches—content-based, collaborative filtering, and hybrid
models—to determine what works best for your audience. More advanced techniques like matrix factorization and neural collaborative filtering can help 
uncover deeper relationships within the data. We also suggest making the system time-aware so that recommendations stay fresh and relevant as trends
shift. Once the models are built, A/B testing with real users will help confirm improvements in engagement and satisfaction. Finally, continuous 
monitoring, regular updates, and a clear dashboard will ensure the recommendation engine remains accurate, transparent, and aligned with your business
needs over time.'''

'Conclusion and Recommendations\nOverall, our analysis has given us a clear and solid foundation for building a strong recommendation system. We started by cleaning and transforming \nthe dataset, converting all categorical values into meaningful numbers and applying scaling so that every feature is treated fairly. Through our \nexploratory analysis, we gained a deeper understanding of what truly influences user choices—such as course ratings, difficulty levels, and learning\nformat—helping us see the patterns that matter. With this groundwork in place, we now have a well-prepared dataset ready for machine learning models.\nMoving forward, we recommend enriching the system with real user behavior signals like clicks, time spent, and course completion, as these reflect \ngenuine user interest. It will be valuable to experiment with multiple modeling approaches—content-based, collaborative filtering, and hybrid\nmodels—to determine what works best for your audience. More advanced techniq

<!-- Model Building Code (Mandatory) -->

In [ ]:
# !pip install scikit-surprise


Final Conclusion

Our collaborative filtering model successfully recommends personalized courses to users based on rating similarity.
The system uses:

User–course rating matrix

Cosine similarity between users

Weighted rating prediction

Top-N recommendations

Model performance was evaluated using Precision@5 and Recall@5, which are standard metrics for recommender systems.
This ensures that the recommendations are both accurate and relevant for users.

The model is now ready for real-time deployment using Streamlit.